In [1]:
import polars as pl
from argparse import ArgumentParser
from pathlib import Path
from prep_data import *
from utils import get_memory_usage

In [2]:
print(f"Memory usage: {get_memory_usage():.2f} MB")


Memory usage: 77.96 MB


In [3]:
players_df = get_players_df()
plays_df = get_plays_df()
tracking_df = get_tracking_df()
print(f"Memory usage: {get_memory_usage():.2f} MB")

Memory usage: 4555.25 MB


In [4]:


tracking_df = add_features_to_tracking_df(tracking_df, players_df, plays_df)
del players_df, plays_df
tracking_df = convert_tracking_to_cartesian(tracking_df)
tracking_df = standardize_tracking_directions(tracking_df)
tracking_df = augment_mirror_tracking(tracking_df)
print(f"Memory usage: {get_memory_usage():.2f} MB")


Memory usage: 8841.74 MB


In [5]:
rel_tracking_df = add_relative_positions(tracking_df)
print(f"Memory usage after adding relative positions: {get_memory_usage():.2f} MB")
del tracking_df  # Free memory
print(f"Memory usage after deleting tracking_df: {get_memory_usage():.2f} MB")


Memory usage after adding relative positions: 14380.21 MB
Memory usage after deleting tracking_df: 11311.87 MB


In [6]:
tkl_loc_tgt_df, rel_tracking_df = get_tackle_loc_target_df(rel_tracking_df)
print(f"Memory usage: {get_memory_usage():.2f} MB")


Lost 0.040% plays when joining with tackle_loc_df
Memory usage: 11453.57 MB


In [ ]:
split_dfs = split_train_test_val(rel_tracking_df, tkl_loc_tgt_df)
del rel_tracking_df, tkl_loc_tgt_df
print(f"Memory usage after splitting: {get_memory_usage():.2f} MB")

Total set: 24956 plays, 1059214 frames
Train set: 17470 plays, 740590 frames
Test set: 3742 plays, 160124 frames
Validation set: 3744 plays, 158500 frames


In [8]:
out_dir = Path("data/split_prepped_data/")
out_dir.mkdir(exist_ok=True, parents=True)

for key, df in split_dfs.items():
    sort_keys = ["gameId", "playId", "mirrored", "frameId"]
    df.sort(sort_keys).write_parquet(out_dir / f"{key}.parquet")
